# 🎨 MangaGen - AI Manga Generation Pipeline

Generate complete manga pages with **consistent characters** from text prompts!

## 📂 Drive Structure (Auto-Created)
```
MyDrive/MangaGen/
├── Models/        # Cached SDXL (~10GB, one-time)
├── Projects/      # Your manga projects
│   └── [project]/
│       ├── scene_plan.json
│       ├── panels/
│       └── output/
└── CharacterLibrary/  # Reusable refs
```

## Setup
1. **Enable GPU**: Runtime → Change runtime type → T4 GPU
2. **Add API Key**: Click 🔑 → Add `GEMINI_API_KEY`
3. **Run All Cells**: Runtime → Run all

---

In [ ]:
#@title 🔧 Cell 1: Setup & Mount Drive
#@markdown This creates organized folders in your Google Drive.

import os
import sys

# ============================================
# Mount Google Drive
# ============================================
print("📁 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# ============================================
# Create Organized Folder Structure
# ============================================
DRIVE_BASE = '/content/drive/MyDrive/MangaGen'
FOLDERS = {
    'models': f'{DRIVE_BASE}/Models',
    'projects': f'{DRIVE_BASE}/Projects',
    'characters': f'{DRIVE_BASE}/CharacterLibrary',
}

print("📂 Creating folder structure...")
for name, path in FOLDERS.items():
    os.makedirs(path, exist_ok=True)
    print(f"   ✅ {path}")

# Set cache directories
os.environ['HF_HOME'] = FOLDERS['models']
os.environ['TRANSFORMERS_CACHE'] = FOLDERS['models']
os.environ['DIFFUSERS_CACHE'] = FOLDERS['models']

# ============================================
# Check GPU
# ============================================
import torch
print("")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU: {gpu_name} ({gpu_mem:.1f} GB)")
else:
    print("❌ No GPU! Runtime → Change runtime type → T4 GPU")
    raise RuntimeError("GPU required!")

# ============================================
# Clone Repository
# ============================================
print("")
print("📦 Cloning repository...")
!rm -rf /content/manga-gen
!git clone --branch mvp/kaggle-flux https://github.com/Barun-2005/manga-gen-ai-pipeline.git /content/manga-gen --quiet
%cd /content/manga-gen
print("✅ Repository cloned")

print("")
print("🎉 Setup complete! Drive folders ready.")

In [ ]:
#@title 📚 Cell 2: Install Dependencies
#@markdown Takes ~2-3 minutes first time.

print("📦 Installing dependencies...")
print("   (This is cleaner than Kaggle - no conflicts!)")
print("")

# Install core dependencies
!pip install -q \
    diffusers==0.30.3 \
    transformers>=4.41.0 \
    accelerate>=0.30.0 \
    safetensors>=0.4.0 \
    google-generativeai>=0.5.0 \
    pydantic>=2.0.0 \
    opencv-python-headless>=4.8.0 \
    Pillow>=10.0.0 \
    reportlab>=4.0.0 \
    python-dotenv>=1.0.0 \
    tqdm>=4.65.0 \
    xformers

# Verify installations
print("")
print("🔍 Verifying...")
import diffusers, transformers, torch
print(f"✅ diffusers: {diffusers.__version__}")
print(f"✅ transformers: {transformers.__version__}")
print(f"✅ torch: {torch.__version__}")

# Critical test: Can we import SDXL pipeline?
from diffusers import StableDiffusionXLPipeline
print("✅ SDXL Pipeline: importable!")

print("")
print("🎉 Dependencies ready!")

In [ ]:
#@title 🔑 Cell 3: API Key Setup

import os

# Try Colab secrets first
try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')
    if api_key:
        os.environ['GEMINI_API_KEY'] = api_key
        print(f"✅ API key loaded from secrets: {api_key[:8]}...")
    else:
        raise ValueError("No key in secrets")
except:
    from getpass import getpass
    api_key = getpass("🔑 Enter Gemini API key: ")
    os.environ['GEMINI_API_KEY'] = api_key
    print("✅ API key set manually")

In [ ]:
#@title ⚙️ Cell 4: Configuration

#@markdown ### 📝 Story Prompt
STORY_PROMPT = "Astra, a space scavenger with silver hair and orange jumpsuit, explores a derelict spaceship and discovers a glowing artifact." #@param {type:"string"}

#@markdown ### 🎨 Visual Style
STYLE = "bw_manga" #@param ["bw_manga", "color_anime"]
LAYOUT = "2x2" #@param ["2x2", "vertical_webtoon", "3_panel", "single"]

#@markdown ### ⚡ Quality Settings
INFERENCE_STEPS = 25 #@param {type:"slider", min:15, max:50, step:5}
GUIDANCE_SCALE = 7.5 #@param {type:"slider", min:5, max:12, step:0.5}

#@markdown ### 📁 Project Name (for organization)
PROJECT_NAME = "my_manga" #@param {type:"string"}

# Create project folder
import os
PROJECT_DIR = f"/content/drive/MyDrive/MangaGen/Projects/{PROJECT_NAME}"
os.makedirs(f"{PROJECT_DIR}/panels", exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/output", exist_ok=True)

print("📋 Configuration:")
print(f"   Project: {PROJECT_NAME}")
print(f"   Style: {STYLE}")
print(f"   Layout: {LAYOUT}")
print(f"   Steps: {INFERENCE_STEPS}")
print(f"")
print(f"📂 Project folder: {PROJECT_DIR}")

In [ ]:
#@title 📝 Cell 5: Generate Scene Plan (Gemini)

import os
os.chdir('/content/manga-gen')

!python scripts/generate_scene_json.py "{STORY_PROMPT}" \
    --style {STYLE} \
    --layout {LAYOUT} \
    --output scene_plan.json

# Copy to project folder
import shutil
shutil.copy('scene_plan.json', f'{PROJECT_DIR}/scene_plan.json')

# Display
import json
if os.path.exists('scene_plan.json'):
    with open('scene_plan.json', 'r') as f:
        scene = json.load(f)
    print(f"\n✅ Scene: {scene.get('title')}")
    print(f"   Panels: {len(scene.get('panels', []))}")
    print(f"   Saved to: {PROJECT_DIR}/scene_plan.json")

In [ ]:
#@title 🎨 Cell 6: Generate Panel Images (SDXL on GPU)
#@markdown ⏱️ First run: ~5-8 min (downloads models to Drive)
#@markdown ⏱️ Cached runs: ~3-5 min

import os
import time

os.chdir('/content/manga-gen')
os.makedirs('outputs', exist_ok=True)

print("🎨 Generating panels with SDXL...")
print(f"   Models cached to: {os.environ.get('HF_HOME')}")
print("")

start = time.time()

!python scripts/generate_panels.py \
    --scene scene_plan.json \
    --output outputs/ \
    --steps {INFERENCE_STEPS} \
    --guidance {GUIDANCE_SCALE}

elapsed = time.time() - start
print(f"\n⏱️ Time: {elapsed/60:.1f} minutes")

# Copy panels to project folder
import shutil
import glob
for f in glob.glob('outputs/*.png'):
    shutil.copy(f, f"{PROJECT_DIR}/panels/")

if elapsed > 60:
    print("✅ Real SDXL images generated!")
else:
    print("⚠️ Fast run - might be mock mode. Check for errors above.")

In [ ]:
#@title 🖼️ Cell 7: Display Generated Panels

from IPython.display import display, Image as IPImage
import glob
import os

os.chdir('/content/manga-gen')

panels = sorted([p for p in glob.glob('outputs/panel_*.png') if 'with_bubbles' not in p])

if panels:
    print("🖼️ Generated Panels:")
    for p in panels:
        print(f"\n{os.path.basename(p)}")
        display(IPImage(filename=p, width=400))
else:
    print("❌ No panels found. Check Cell 6 for errors.")

In [ ]:
#@title 💬 Cell 8: Place Bubbles + Compose Page

import os
os.chdir('/content/manga-gen')

print("💬 Placing dialogue bubbles...")
!python scripts/place_bubbles.py \
    --panels outputs/ \
    --scene scene_plan.json \
    --output bubbles.json

print("\n📄 Composing final page...")
!python scripts/compose_page.py \
    --panels outputs/ \
    --bubbles bubbles.json \
    --scene scene_plan.json \
    --output outputs/

# Copy outputs to project folder
import shutil
for f in ['outputs/manga_page.png', 'outputs/manga_page.pdf']:
    if os.path.exists(f):
        shutil.copy(f, f"{PROJECT_DIR}/output/")
if os.path.exists('manga_output.zip'):
    shutil.copy('manga_output.zip', f"{PROJECT_DIR}/output/")

# Display
if os.path.exists('outputs/manga_page.png'):
    from IPython.display import display, Image as IPImage
    print("\n🎨 Final Manga Page:")
    display(IPImage(filename='outputs/manga_page.png', width=600))
    print(f"\n📂 Saved to: {PROJECT_DIR}/output/")

In [ ]:
#@title 📥 Cell 9: Download

import os
from google.colab import files

os.chdir('/content/manga-gen')

print("🎉 YOUR MANGA IS READY!")
print("="*50)
print(f"\n📂 Files saved to Google Drive:")
print(f"   {PROJECT_DIR}/output/")
print(f"")
print("📥 Or download directly:")

if os.path.exists('manga_output.zip'):
    files.download('manga_output.zip')
else:
    print("   Check your Drive: MangaGen/Projects/" + PROJECT_NAME)

---

## 📊 Summary

| Step | Time |
|------|------|
| Setup | ~30s |
| Install | ~2 min |
| Scene (Gemini) | ~5 sec |
| Panels (SDXL) | ~4 min |
| Bubbles + PDF | ~15 sec |
| **Total** | **~7 min** |

---

Made with ❤️ by Barun